Collecting Data

In [ ]:
import pandas as pd
import requests as r
import datetime

In [ ]:
# modules and code to upload clean tables to DBeaver
import sql_functions as sf
import psycopg2

In [ ]:
schema = 'capstone_anglianwater'
engine = sf.get_engine()

UK water supply price (1988-2023)

In [ ]:
#read csv file
water_supply_columns = ['date', 'value']
water_supply = pd.read_csv('/Users/stellaxingli/neuefische/Tap_vs_Bottle/data/water_supply_price_UK.csv',
                     names=water_supply_columns, 
                     skiprows = 188)

"CPI INDEX 04.4.1 : WATER SUPPLY 2015=100" means that the Consumer Price Index for water supply was calculated in such a way that its value was set to 100 in the year 2015, and subsequent values represent changes in the price level of water supply relative to that base year. 

For example, if the index is 110 in a certain year, it would mean that water supply costs 10% more than it did in 2015.

In [ ]:
# change the column 'period' into datetime format
water_supply['date'] = pd.to_datetime(water_supply['date'])


In [ ]:
water_supply= water_supply.replace(',','.', regex=True)

In [9]:
# get the info and first 5 rows from the file
print(water_supply.info())
print(water_supply.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    432 non-null    datetime64[ns]
 1   value   432 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 6.9 KB
None
        date  value
0 1988-01-01   21.2
1 1988-02-01   21.2
2 1988-03-01   21.2
3 1988-04-01   22.5
4 1988-05-01   22.5


In [ ]:
table_name = 'water_supply_price'
if engine!=None:
    try:
        water_supply.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print('No engine')